요거 완전히 틀린 접근!

In [106]:
# module import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop

In [86]:
# path 설정
project_path = "/content/drive/My Drive/TOY/Data Analysis/Kaggle/novel-COVID"
data_path = f"{project_path}/data"

In [87]:
# load data
df = pd.read_csv(f"{data_path}/covid_19_data.csv")
covid = df.copy()

In [88]:
# 데이터프레임 확인
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53927 entries, 0 to 53926
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              53927 non-null  int64  
 1   ObservationDate  53927 non-null  object 
 2   Province/State   32870 non-null  object 
 3   Country/Region   53927 non-null  object 
 4   Last Update      53927 non-null  object 
 5   Confirmed        53927 non-null  float64
 6   Deaths           53927 non-null  float64
 7   Recovered        53927 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 3.3+ MB


In [89]:
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [90]:
covid.tail() # 누적인듯 하지?

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
53922,53923,06/30/2020,Zacatecas,Mexico,2020-07-01 04:33:52,908.0,96.0,626.0
53923,53924,06/30/2020,Zakarpattia Oblast,Ukraine,2020-07-01 04:33:52,2889.0,91.0,943.0
53924,53925,06/30/2020,Zaporizhia Oblast,Ukraine,2020-07-01 04:33:52,572.0,17.0,418.0
53925,53926,06/30/2020,Zhejiang,Mainland China,2020-07-01 04:33:52,1269.0,1.0,1267.0
53926,53927,06/30/2020,Zhytomyr Oblast,Ukraine,2020-07-01 04:33:52,1404.0,29.0,746.0


In [91]:
# 결측치?
covid.isnull().sum()

SNo                    0
ObservationDate        0
Province/State     21057
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64

# 1. 누적이 뭘 기준으로 누적인가?

* 날짜도 날짜인데, Country/Region -> Province/State 인듯하다.
* ㅇㅋ. 그러면 날짜별 -> Country/Region별 -> Province/State별로 묶어서 누적을 해체해보는 건 어떤가?
* 그런데, 그러면 Province/State가 결측치인 곳은 어떡하나?
* 날짜별 -> Country/Region별로 풀어헤져야겠다!

In [120]:
covid[80:120]

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
80,81,01/23/2020,NaN,Australia,1/23/20 17:00,0.0,0.0,0.0
81,82,01/23/2020,NaN,Mexico,1/23/20 17:00,0.0,0.0,0.0
82,83,01/23/2020,NaN,Brazil,1/23/20 17:00,0.0,0.0,0.0
83,84,01/23/2020,NaN,Colombia,1/23/20 17:00,0.0,0.0,0.0
84,85,01/24/2020,Hubei,Mainland China,1/24/20 17:00,549.0,24.0,31.0
85,86,01/24/2020,Guangdong,Mainland China,1/24/20 17:00,53.0,0.0,2.0
86,87,01/24/2020,Zhejiang,Mainland China,1/24/20 17:00,43.0,0.0,1.0
87,88,01/24/2020,Beijing,Mainland China,1/24/20 17:00,36.0,0.0,1.0
88,89,01/24/2020,Chongqing,Mainland China,1/24/20 17:00,27.0,0.0,0.0
89,90,01/24/2020,Hunan,Mainland China,1/24/20 17:00,24.0,0.0,0.0


In [8]:
# 일단 그르면 NaN인 곳을 찾아보자.
null_countries = covid[covid['Province/State'].isnull()]['Country/Region'].unique() # 여기 있는 애들이 전부 다 province/state 없으면 그냥 묶어 버리면 된다.

for nc in null_countries:
    if str(covid[covid['Country/Region'] == nc]['Province/State']) != 'NaN':
        print(nc)
        break

Japan


# 1. 사망자

In [93]:
covid_death = covid.groupby(by=['ObservationDate', 'Country/Region'])['Deaths'].max()
cum_death = covid_death.unstack().reset_index()
cum_death = cum_death.fillna(0.0)
display(cum_death)
cum_death_arr = cum_death.sum(axis=1)
print(cum_death_arr)
death_by_date = cum_death_arr.copy()
for i in range(1, len(cum_death_arr)):
    death_by_date[i] = cum_death_arr[i] - cum_death_arr[i-1]
print(death_by_date)

Country/Region,ObservationDate,Azerbaijan,"('St. Martin',)",Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,"Bahamas, The",Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,...,Slovenia,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,St. Martin,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,The Bahamas,The Gambia,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,UK,US,Uganda,Ukraine,United Arab Emirates,Uruguay,Uzbekistan,Vatican City,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe,occupied Palestinian territory
0,01/22/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01/23/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01/24/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01/25/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01/26/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,06/26/2020,0.0,0.0,683.0,51.0,885.0,52.0,10.0,3.0,1184.0,410.0,0.0,49.0,698.0,187.0,11.0,0.0,73.0,1661.0,7.0,373.0,9731.0,2.0,14.0,0.0,934.0,178.0,1.0,13966.0,3.0,215.0,53.0,6.0,1.0,10.0,0.0,313.0,5448.0,0.0,0.0,...,109.0,90.0,2340.0,282.0,36.0,8418.0,11.0,0.0,572.0,10.0,2298.0,1962.0,8.0,7.0,52.0,21.0,58.0,0.0,0.0,0.0,14.0,8.0,50.0,5065.0,38888.0,31342.0,0.0,178.0,310.0,26.0,20.0,0.0,41.0,0.0,3.0,1.0,293.0,21.0,6.0,0.0
157,06/27/2020,0.0,0.0,703.0,53.0,892.0,52.0,10.0,3.0,1207.0,421.0,0.0,49.0,700.0,193.0,11.0,0.0,78.0,1695.0,7.0,377.0,9732.0,2.0,14.0,0.0,970.0,178.0,1.0,14263.0,3.0,216.0,53.0,6.0,1.0,12.0,0.0,313.0,5448.0,0.0,0.0,...,109.0,90.0,2413.0,282.0,36.0,8420.0,11.0,0.0,572.0,11.0,2298.0,1962.0,9.0,7.0,52.0,21.0,58.0,0.0,0.0,0.0,14.0,8.0,50.0,5082.0,38982.0,31368.0,0.0,180.0,311.0,26.0,20.0,0.0,42.0,0.0,4.0,1.0,296.0,21.0,6.0,0.0
158,06/28/2020,0.0,0.0,721.0,55.0,897.0,52.0,11.0,3.0,1232.0,426.0,0.0,49.0,702.0,198.0,11.0,0.0,83.0,1738.0,7.0,383.0,9732.0,2.0,16.0,0.0,1014.0,178.0,1.0,14338.0,3.0,219.0,53.0,6.0,1.0,12.0,0.0,313.0,5448.0,0.0,0.0,...,111.0,90.0,2456.0,282.0,36.0,8421.0,11.0,0.0,572.0,11.0,2298.0,1962.0,9.0,7.0,52.0,21.0,58.0,0.0,0.0

0          17.0
1          17.0
2          24.0
3          40.0
4          52.0
         ...   
156    258555.0
157    260539.0
158    262115.0
159    263824.0
160    266768.0
Length: 161, dtype: float64
0        17.0
1         0.0
2         7.0
3        16.0
4        12.0
        ...  
156    2196.0
157    1984.0
158    1576.0
159    1709.0
160    2944.0
Length: 161, dtype: float64


# 2. 확진자

In [95]:
covid_confirmed = covid.groupby(by=['ObservationDate', 'Country/Region'])['Confirmed'].max()
cum_confirmed = covid_confirmed.unstack().reset_index()
cum_confirmed = cum_confirmed.fillna(0.0)
display(cum_confirmed)
cum_confirmed_arr = cum_confirmed.sum(axis=1)
print(cum_confirmed_arr)
confirmed_by_date = cum_confirmed_arr.copy()
for i in range(1, len(cum_confirmed_arr)):
    confirmed_by_date[i] = cum_confirmed_arr[i] - cum_confirmed_arr[i-1]
print(confirmed_by_date)

Country/Region,ObservationDate,Azerbaijan,"('St. Martin',)",Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,"Bahamas, The",Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,...,Slovenia,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,St. Martin,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,The Bahamas,The Gambia,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,UK,US,Uganda,Ukraine,United Arab Emirates,Uruguay,Uzbekistan,Vatican City,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe,occupied Palestinian territory
0,01/22/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01/23/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01/24/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01/25/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01/26/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,06/26/2020,0.0,0.0,30451.0,2269.0,12685.0,855.0,212.0,65.0,55343.0,23247.0,0.0,3174.0,17522.0,15369.0,104.0,0.0,24805.0,130474.0,97.0,60713.0,61106.0,24.0,1053.0,70.0,29423.0,3935.0,92.0,258508.0,141.0,4513.0,941.0,293.0,144.0,1027.0,139.0,12592.0,55079.0,0.0,0.0,...,1558.0,2878.0,124590.0,12653.0,1942.0,71632.0,2014.0,0.0,9257.0,389.0,19164.0,31486.0,255.0,447.0,5747.0,509.0,3162.0,0.0,0.0,24.0,591.0,124.0,1164.0,194511.0,159958.0,391220.0,833.0,4836.0,46973.0,919.0,7427.0,0.0,4779.0,353.0,1557.0,10.0,1089.0,1531.0,561.0,0.0
157,06/27/2020,0.0,0.0,30616.0,2330.0,12968.0,855.0,259.0,65.0,57744.0,23909.0,0.0,3177.0,17580.0,15890.0,104.0,0.0,25267.0,133978.0,97.0,61095.0,61209.0,24.0,1124.0,75.0,30676.0,3935.0,92.0,265581.0,141.0,4625.0,941.0,296.0,170.0,1091.0,141.0,12592.0,55079.0,0.0,0.0,...,1572.0,2878.0,131800.0,12715.0,1942.0,71839.0,2033.0,0.0,9257.0,467.0,19164.0,31555.0,256.0,447.0,5799.0,509.0,3162.0,0.0,0.0,24.0,615.0,126.0,1168.0,195883.0,160150.0,391923.0,848.0,5004.0,47360.0,924.0,7682.0,0.0,5130.0,355.0,1815.0,10.0,1103.0,1531.0,567.0,0.0
158,06/28/2020,0.0,0.0,30967.0,2402.0,13273.0,855.0,267.0,69.0,59933.0,24645.0,0.0,3184.0,17654.0,16424.0

0          452.0
1          458.0
2          573.0
3          799.0
4         1111.0
         ...    
156    4645094.0
157    4714756.0
158    4782358.0
159    4845366.0
160    4913108.0
Length: 161, dtype: float64
0        452.0
1          6.0
2        115.0
3        226.0
4        312.0
        ...   
156    73938.0
157    69662.0
158    67602.0
159    63008.0
160    67742.0
Length: 161, dtype: float64


# 3. 완치자

In [97]:
covid_recovered = covid.groupby(by=['ObservationDate', 'Country/Region'])['Recovered'].max()
cum_recovered = covid_recovered.unstack().reset_index()
cum_recovered = cum_recovered.fillna(0.0)
display(cum_recovered)
cum_recovered_arr = cum_recovered.sum(axis=1)
print(cum_recovered_arr)
recovered_by_date = cum_recovered_arr.copy()
for i in range(1, len(cum_recovered_arr)):
    recovered_by_date[i] = cum_recovered_arr[i] - cum_recovered_arr[i-1]
print(recovered_by_date)

Country/Region,ObservationDate,Azerbaijan,"('St. Martin',)",Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,"Bahamas, The",Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,...,Slovenia,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,St. Martin,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,The Bahamas,The Gambia,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,UK,US,Uganda,Ukraine,United Arab Emirates,Uruguay,Uzbekistan,Vatican City,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe,occupied Palestinian territory
0,01/22/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01/23/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01/24/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01/25/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01/26/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,06/26/2020,0.0,0.0,10306.0,1298.0,9066.0,799.0,81.0,22.0,18416.0,12149.0,0.0,2784.0,16348.0,8364.0,84.0,0.0,19137.0,53133.0,90.0,42689.0,16918.0,18.0,292.0,38.0,7736.0,2338.0,25.0,87850.0,138.0,2457.0,830.0,215.0,93.0,568.0,129.0,10100.0,31210.0,0.0,0.0,...,1376.0,868.0,64111.0,11317.0,224.0,40736.0,1619.0,0.0,4014.0,181.0,0.0,29000.0,102.0,435.0,4331.0,183.0,3040.0,0.0,0.0,24.0,395.0,109.0,1023.0,167198.0,512.0,670809.0,761.0,1800.0,35469.0,818.0,5038.0,0.0,1327.0,330.0,446.0,8.0,402.0,1233.0,135.0,0.0
157,06/27/2020,0.0,0.0,10674.0,1346.0,9202.0,799.0,81.0,22.0,19143.0,12911.0,0.0,2784.0,16371.0,8719.0,87.0,0.0,19781.0,54318.0,90.0,44126.0,16941.0,18.0,295.0,38.0,8158.0,2338.0,25.0,88784.0,138.0,2475.0,830.0,216.0,115.0,568.0,129.0,10100.0,31404.0,0.0,0.0,...,1376.0,868.0,67094.0,11364.0,224.0,40736.0,1639.0,0.0,4014.0,191.0,0.0,29100.0,102.0,435.0,4391.0,183.0,3053.0,0.0,0.0,24.0,396.0,109.0,1025.0,169182.0,512.0,679308.0,761.0,1853.0,35834.0,818.0,5240.0,0.0,1327.0,330.0,446.0,8.0,417.0,1233.0,142.0,0.0
158,06/28/2020,0.0,0.0,12604.0,1384.0,9371.0,799.0,81.0,22.0,20134.0,13116.0,0.0,2784.0,16401.0,9026.0,87.0,0.0,20517.0,55727.0,90.0,45027.0,16941.0,1

0           28.0
1           28.0
2           31.0
3           32.0
4           45.0
         ...    
156    3234604.0
157    3289724.0
158    3340195.0
159    3356569.0
160    3417505.0
Length: 161, dtype: float64
0         28.0
1          0.0
2          3.0
3          1.0
4         13.0
        ...   
156    63105.0
157    55120.0
158    50471.0
159    16374.0
160    60936.0
Length: 161, dtype: float64


# 4. 데이터 합치기

In [114]:
confirmed_by_date[confirmed_by_date < 0]

113   -362617.0
119   -199581.0
127    -11271.0
131   -192229.0
140   -202764.0
141    -69571.0
dtype: float64

In [110]:
raw_data = pd.DataFrame([confirmed_by_date, recovered_by_date, death_by_date]).T.values
raw_data

array([[ 4.52000e+02,  2.80000e+01,  1.70000e+01],
       [ 6.00000e+00,  0.00000e+00,  0.00000e+00],
       [ 1.15000e+02,  3.00000e+00,  7.00000e+00],
       [ 2.26000e+02,  1.00000e+00,  1.60000e+01],
       [ 3.12000e+02,  1.30000e+01,  1.20000e+01],
       [ 3.72000e+02,  3.00000e+00,  2.40000e+01],
       [ 2.15000e+03,  3.80000e+01,  4.90000e+01],
       [ 1.10000e+01,  8.00000e+00,  0.00000e+00],
       [ 1.36100e+03,  4.00000e+00,  3.70000e+01],
       [ 9.33000e+02,  5.10000e+01,  4.20000e+01],
       [ 1.36700e+03,  2.80000e+01,  4.50000e+01],
       [ 4.03700e+03,  1.27000e+02,  1.02000e+02],
       [ 2.35000e+03,  9.10000e+01,  6.40000e+01],
       [ 3.17800e+03,  1.36000e+02,  6.60000e+01],
       [ 3.00000e+03,  1.11000e+02,  7.00000e+01],
       [ 2.48400e+03,  1.86000e+02,  6.90000e+01],
       [ 2.89300e+03,  3.00000e+02,  8.10000e+01],
       [ 2.17300e+03,  3.32000e+02,  8.10000e+01],
       [ 2.54900e+03,  3.60000e+02,  9.10000e+01],
       [ 2.19300e+03,  4.30000e

In [103]:
data = pd.DataFrame([confirmed_by_date, recovered_by_date, death_by_date]).T.values
print(data)
print(data.shape)

[[ 4.52000e+02  2.80000e+01  1.70000e+01]
 [ 6.00000e+00  0.00000e+00  0.00000e+00]
 [ 1.15000e+02  3.00000e+00  7.00000e+00]
 [ 2.26000e+02  1.00000e+00  1.60000e+01]
 [ 3.12000e+02  1.30000e+01  1.20000e+01]
 [ 3.72000e+02  3.00000e+00  2.40000e+01]
 [ 2.15000e+03  3.80000e+01  4.90000e+01]
 [ 1.10000e+01  8.00000e+00  0.00000e+00]
 [ 1.36100e+03  4.00000e+00  3.70000e+01]
 [ 9.33000e+02  5.10000e+01  4.20000e+01]
 [ 1.36700e+03  2.80000e+01  4.50000e+01]
 [ 4.03700e+03  1.27000e+02  1.02000e+02]
 [ 2.35000e+03  9.10000e+01  6.40000e+01]
 [ 3.17800e+03  1.36000e+02  6.60000e+01]
 [ 3.00000e+03  1.11000e+02  7.00000e+01]
 [ 2.48400e+03  1.86000e+02  6.90000e+01]
 [ 2.89300e+03  3.00000e+02  8.10000e+01]
 [ 2.17300e+03  3.32000e+02  8.10000e+01]
 [ 2.54900e+03  3.60000e+02  9.10000e+01]
 [ 2.19300e+03  4.30000e+02  1.03000e+02]
 [ 1.65600e+03  4.36000e+02  9.40000e+01]
 [ 4.70000e+01  7.00000e+01  0.00000e+00]
 [ 1.48530e+04  7.80000e+02  2.43000e+02]
 [ 6.25700e+03  1.31800e+03  1.470

# 5. 표준화

In [104]:
scaler = StandardScaler()
data = scaler.fit_transform(data)
data

array([[-5.64480953e-01, -5.23249963e-01, -4.21001013e-01],
       [-5.72854983e-01, -5.23941089e-01, -4.25365196e-01],
       [-5.70808415e-01, -5.23867040e-01, -4.23568180e-01],
       [-5.68724295e-01, -5.23916406e-01, -4.21257729e-01],
       [-5.67109572e-01, -5.23620209e-01, -4.22284596e-01],
       [-5.65983021e-01, -5.23867040e-01, -4.19203996e-01],
       [-5.32599556e-01, -5.23003133e-01, -4.12786079e-01],
       [-5.72761104e-01, -5.23743625e-01, -4.25365196e-01],
       [-5.47413703e-01, -5.23842357e-01, -4.15866679e-01],
       [-5.55449768e-01, -5.22682253e-01, -4.14583096e-01],
       [-5.47301048e-01, -5.23249963e-01, -4.13812946e-01],
       [-4.97169523e-01, -5.20806339e-01, -3.99180094e-01],
       [-5.28844385e-01, -5.21694930e-01, -4.08935328e-01],
       [-5.13297980e-01, -5.20584192e-01, -4.08421895e-01],
       [-5.16640081e-01, -5.21201268e-01, -4.07395028e-01],
       [-5.26328421e-01, -5.19350038e-01, -4.07651745e-01],
       [-5.18649097e-01, -5.16536168e-01

# 5. LSTM 모델링

In [108]:
# split data
X_data = data[:, :2]
y_data = data[:, -1]
print(f"X_data: {X_data.shape}, y_data: {y_data.shape}")

X_data: (161, 2), y_data: (161,)


In [ ]:
# 테스트 모델링
